In [2]:
import pandas as pd 
import numpy as np

In [16]:
ICE2 = pd.read_csv("ICE24051.csv")
ICE2

,eval,comment,category
0,0,teacher are punctual but they should also give...,teaching
1,1,Good,teaching
2,1,Excellent lectures are delivered by teachers a...,teaching
3,1,Good,teaching
4,1,teachers give us all the information required ...,teaching
...,...,...,...
1105,1,they all are held in super,extracurricular
1106,1,good,extracurricular
1107,1,the extracurricular activities held in univers...,extracurricular
1108,1,Our university has lot of extracurricular goin...,extracurricular


In [17]:
import re

In [20]:
import string

In [21]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[0-9]+','', text)
    return text

In [22]:
ICE2['comment'] = ICE2['comment'].map(clean_text)
ICE2

,eval,comment,category
0,0,teacher are punctual but they should also give...,teaching
1,1,good,teaching
2,1,excellent lectures are delivered by teachers a...,teaching
3,1,good,teaching
4,1,teachers give us all the information required ...,teaching
...,...,...,...
1105,1,they all are held in super,extracurricular
1106,1,good,extracurricular
1107,1,the extracurricular activities held in univers...,extracurricular
1108,1,our university has lot of extracurricular goin...,extracurricular


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')

commentCV = cv.fit_transform(ICE2['comment'])
commentCV_dtm = pd.DataFrame(commentCV.toarray(), columns = cv.get_feature_names())

commentCV_dtm.index = ICE2['comment'].index
commentCV_dtm

,abilities,ability,able,abroad,absolutely,absurd,abt,academic,accessable,accitivties,...,works,world,worth,write,writing,wrong,yeah,year,years,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
ICE2.groupby(['eval','category']).count().unstack()

comment                                                         \
category coursecontent examination extracurricular labwork library_facilities   
eval                                                                            
-1                  30          24              12      37                 31   
 0                  27          31              19      16                 24   
 1                 128         130             154     132                130   

                   
category teaching  
eval               
-1             13  
 0             35  
 1            137

In [29]:
totalCT = commentCV_dtm.sum()
commentCV_dtm[totalCT.sort_values(ascending = False).index[:20]].sum()

good          654
excellent      74
students       62
university     61
library        48
books          47
course         43
pattern        40
lab            39
teachers       39
activities     37
knowledge      36
time           32
content        31
teaching       31
paper          30
checking       30
work           30
average        29
exam           29
dtype: int64

In [33]:
conda install -c conda-forge gensim


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\Sivakon.M\Anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [34]:
from gensim import matutils, models
import scipy.sparse  

In [35]:
tdm = commentCV_dtm.transpose()
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v, p) for p, v in cv.vocabulary_.items())

In [36]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=20)
lda.print_topics()

[(0,
  '0.293*"good" + 0.018*"university" + 0.016*"activities" + 0.015*"course" + 0.013*"students" + 0.013*"delivery" + 0.011*"courses" + 0.011*"teaching" + 0.011*"lecture" + 0.010*"interaction"'),
 (1,
  '0.062*"excellent" + 0.021*"university" + 0.017*"best" + 0.015*"time" + 0.012*"bad" + 0.011*"course" + 0.010*"students" + 0.010*"facilities" + 0.009*"required" + 0.009*"library"'),
 (2,
  '0.029*"books" + 0.027*"good" + 0.025*"pattern" + 0.023*"lab" + 0.021*"library" + 0.019*"paper" + 0.019*"checking" + 0.018*"average" + 0.017*"knowledge" + 0.017*"exam"')]

In [39]:
from sklearn.model_selection import train_test_split

Xs_docs = ICE2['comment']
Ys_evals = ICE2['eval']
xs_training, xs_test, y_training, y_test = train_test_split(Xs_docs, Ys_evals, test_size = 0.2)

In [40]:
from sklearn.naive_bayes import MultinomialNB
cv = CountVectorizer(stop_words='english')
features = cv.fit_transform(xs_training)
model = MultinomialNB()
model.fit(features, y_training)
feature_test = cv.transform(xs_test)
print(model.score(feature_test,y_test))

0.8018018018018018


In [41]:
from sklearn import svm
cv = CountVectorizer(stop_words='english')
features = cv.fit_transform(xs_training)
model = svm.SVC()
model.fit(features, y_training)
feature_test = cv.transform(xs_test)
print(model.score(feature_test,y_test))

0.7837837837837838
